In [1]:
import os

from nltk.tokenize.moses import MosesTokenizer, MosesDetokenizer
from nltk.translate.bleu_score import (sentence_bleu, modified_precision,
    closest_ref_length)
from fractions import Fraction
from collections import Counter

from openpyxl import load_workbook

tokenizer, detokenizer = MosesTokenizer(), MosesDetokenizer()

In [2]:
def readfile(file_loc):
    data = open(file_loc).readlines()
    return [tokenizer.tokenize(datum) for datum in data]

In [3]:
wb = load_workbook('BLEU Template.xlsm', keep_vba=True)
sheet = wb[wb.sheetnames[0]]

In [4]:
sources = readfile('data/sources.data')
references = readfile('data/references.data')
hypotheses = readfile('data/hypotheses.data')

results = []

for source, reference, hypothesis in zip(sources, references, hypotheses):
    set_summary = []
    
    set_summary.append(detokenizer.detokenize(source, return_str=True))
    set_summary.append(detokenizer.detokenize(reference, return_str=True))
    set_summary.append(detokenizer.detokenize(hypothesis, return_str=True))
    
    hyp_len = len(hypothesis)
    ref_len = closest_ref_length([reference], hyp_len)
                                 
    set_summary.append(ref_len)
    set_summary.append(hyp_len)
    
    for i, _ in enumerate((0.25, 0.25, 0.25, 0.25), start=1):
        p_i = modified_precision([reference], hypothesis, i)
        set_summary.append(p_i.numerator)
        set_summary.append(p_i.denominator)

    set_summary.append(sentence_bleu([reference], hypothesis))
    results.append(set_summary)

/mnt/c/ubuntu-bash/BLEU-Evaluation-Tool/venv/lib/python3.5/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/mnt/c/ubuntu-bash/BLEU-Evaluation-Tool/venv/lib/python3.5/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/mnt/c/ubuntu-bash/BLEU-Evaluation-Tool/venv/lib/python3.5/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [5]:
for x in range(3, 3 + len(results)):
    for y in range(1, 15):
        sheet.cell(row=x, column=y).value = results[x - 3][y - 1] 

In [6]:
output_file = 'Evaluation Output.xlsm'

wb.save(output_file)
os.startfile(path, 'open')

AttributeError: module 'os' has no attribute 'startfile'